<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
      AI/ML-Powered Evidence-Based Pathway Analytics for Fraud, Waste, and Abuse (FWA) Detection in Healthcare <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b><br>
<p style = 'font-size:16px;font-family:Arial'>Current FWA(Fraud, waste and abuse) detection processes in healthcare industry rely heavily on traditional queries and manual auditing, which are slow and not scalable. Similarly, identifying cost inefficiencies and poor adherence to Value-Based Care models example in complex musculoskeletal (MSK) cases is difficult due to scattered data and limited advanced analytics. Key challenges include:
<ul style = 'font-size:16px;font-family:Arial'><li>Overutilization of costly procedures not aligned with best-practice medical guidelines. For example: A significant portion of spinal surgeries are conducted without preceding or follow-up physical therapy.</li>
    <li>Underuse of conservative or necessary diagnostic/treatment steps.</li>
    <li>Limited visibility into provider-level behavior and care deviations.  For example: Providers with a history of license suspensions or violations may exhibit patterns of deviation prior to enforcement.</li>
    <li>Ineffective early identification of high-risk patterns that drive up costs and reduce quality.</li>
    </ul>
<p style = 'font-size:16px;font-family:Arial'>
    <b>Proposed Solution</b><br> Evidence-Based Pathway Monitoring Using AI/ML
Implement a robust analytics platform leveraging machine learning, npath analysis, and visual dashboards to detect deviations from evidence based medication(EBM) and reduce fraud,waste and abuse(FWA). With the help of the information in the dashboards we can find out the deviations in the recommended practices like Surgical interventions without appropriate diagnostics or conservative treatments. Cases where diagnostics are skipped or treatment is prematurely escalated. Patient journey flows across diagnostic and treatment steps. Predict patterns of behavior that frequently lead to FWA incidents.
<p style = 'font-size:16px;font-family:Arial'>
    <b>Benefits</b>
<ul style = 'font-size:16px;font-family:Arial'><li>Cost Reduction: Avoid unnecessary procedures and identify FWA through proactive monitoring.</li>
    <li>Better Outcomes: Promote adherence to proven clinical pathways for optimal care quality.</li>
    <li>Operational Efficiency: Reduce manual auditing and investigation efforts.</li>
    <li>Scalability: Apply methodology across departments, specialties, and provider networks.
    <li>Procedure-level cost breakdowns including diagnostics, therapies, and surgeries.</li>
</ul>

<p style = 'font-size:20px;font-family:Arial'><b>Solution Steps</b><br>
<p style = 'font-size:16px;font-family:Arial'>Due to the size and complexity of the solution, we have divided the solution approach into two steps.<ol style = 'font-size:16px;font-family:Arial'>
 <div style="text-align: left; ">
        <a href="./FWA_Dataprocessing_sql.ipynb" style="padding: 10px; background-color: #00233c; color: #FFFFFF; text-decoration: none; border-radius: 5px;">
         Data processing
     </a> </div> <br>This notebook loads the source data and does the data-processing required for the dashboard reporting. The target tables created in the notebook are then be used for the reporting.</p>

<div style="text-align: left; ">
        <a href="https://superset.env.ci.clearscape.teradata.com/superset/dashboard/p/6boOEgZQ8rk/" style="padding: 10px; background-color: #00233c; color: #FFFFFF; text-decoration: none; border-radius: 5px;">
         Superset Dashboards
    </a> </div> <p style = 'font-size:16px;font-family:Arial'><br> The reporting dashboard based on the tables created in using dataprocessing sqls.</p>
    </ol> 
</p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>1. Connect to Vantage, Import python packages and explore the dataset</b></p>


In [ ]:
#import libraries
import getpass
from teradataml import *

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

display.max_rows=5

<p style = 'font-size:16px;font-family:Arial'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=FF_FWA_Reporting_in_Healthcare_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:18px;font-family:Arial'> <b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>

In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_HealthcareFWA_cloud');"
 # takes about 30 seconds, estimated space: 0Mb
#%run -i ../run_procedure.py "call get_data('DEMO_HealthcareFWA_local');" 
# takes about 7minutes, estimated space: 800Mb

<p style = 'font-size:16px;font-family:Arial'>Optional step – We should execute the below step only if we want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>2. Data Exploration </b></p>

<p style = 'font-size:16px;font-family:Arial'>Let us start by listing all tables/datasets available for us. </p>

In [ ]:
db_list_tables('DEMO_HealthcareFWA')

<p style = 'font-size:16px;font-family:Arial'>As we can see from above, we have data in tables about the Procedure, conditions, medications etc. We also have patients billing history e.g in Wc_Sv1header, Wc_Sv1detail. We have information about musculoskeletal, diabetics and maternity cases. </p>

In [ ]:
df_proc = DataFrame(in_schema("DEMO_HealthcareFWA", "Va_Procedures"))
df_proc

In [ ]:
df_sv1_header = DataFrame(in_schema("DEMO_HealthcareFWA", "Wc_Sv1header"))
df_sv1_header

In [ ]:
df_sv1_detail = DataFrame(in_schema("DEMO_HealthcareFWA", "Wc_Sv1detail"))
df_sv1_detail

<p style = 'font-size:16px;font-family:Arial'>You can check the information available in the other tables also. In the <a href = './FWA_Dataprocessing_sql.ipynb'>Dataprocessing Notebook</a>  we can see how we derive meaningful information from these tables and used them in reporting. </p>


<p style = 'font-size:16px;font-family:Arial'>The dasboard looks like below and it uses the tables created in the dataprocessing step.

<center><img src="images/fwa-dashboard.jpg" alt="fwa_dashboard" width=800 height=600/></center>


<p style = 'font-size:16px;font-family:Arial'>
<div style="text-align: left; ">
        <a href="./FWA_Dataprocessing_sql.ipynb" style="padding: 10px; background-color: #00233c; color: #FFFFFF; text-decoration: none; border-radius: 5px;">
         Superset Dashboards
        </a> </div></p> <p style = 'font-size:16px;font-family:Arial'> Please review the insights derived from the source data and assess how they might assist in identifying Fraud, Waste, and Abuse (FWA).</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>3. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_HealthcareFWA');"        # Takes 10 seconds

In [ ]:
remove_context()

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>